In [71]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os
import cv2
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from tensorflow.keras.optimizers import Adam

In [75]:
# Define the base directory path containing the dataset of images.
# The dataset is organized into two categories: 'Mask' and 'No_Mask'.

path1 = r"C:\Mask_NoMask"
cate = ['Mask', 'No_Mask']

In [77]:
image_size = 200 # Set the target image size for resizing all images in the dataset.
input_image = [] # Initialize an empty list to store the image data along with their corresponding labels.

# Loop through each category in the dataset (e.g., 'Mask' and 'No_Mask').
for i in cate:
    folders = os.path.join(path1, i) # Construct the full path to the folder corresponding to the current category.
    label = cate.index(i)   # Assign a numeric label to the category (e.g., 0 for 'Mask', 1 for 'No_Mask').

    # Iterate through all the images in the category's folder.
    for image in os.listdir(folders):
        image_path = os.path.join(folders, image)
        image_array = cv2.imread(image_path)
        image_array = cv2.resize(image_array, (image_size, image_size))
        input_image.append([image_array, label])

In [79]:
input_image[1000]

[array([[[255, 253, 255],
         [255, 253, 255],
         [255, 252, 255],
         ...,
         [ 91, 105, 128],
         [ 79,  93, 116],
         [ 70,  84, 107]],
 
        [[255, 253, 254],
         [255, 253, 255],
         [255, 253, 255],
         ...,
         [ 94, 108, 131],
         [ 88, 102, 125],
         [ 83,  97, 120]],
 
        [[255, 254, 253],
         [255, 254, 253],
         [255, 254, 254],
         ...,
         [ 98, 112, 135],
         [ 99, 113, 136],
         [100, 114, 137]],
 
        ...,
 
        [[251, 255, 252],
         [251, 255, 253],
         [251, 255, 254],
         ...,
         [112, 132, 169],
         [114, 133, 168],
         [116, 135, 168]],
 
        [[252, 255, 252],
         [251, 255, 253],
         [251, 255, 254],
         ...,
         [117, 135, 173],
         [114, 132, 168],
         [112, 131, 165]],
 
        [[253, 255, 252],
         [252, 255, 253],
         [251, 255, 254],
         ...,
         [120, 138, 175],
  

In [25]:
np.random.shuffle(input_image) # Shuffle the input_image list randomly to ensure the data is not ordered by category.

In [81]:
X = []  # Initialize empty lists to store features (X) and labels (Y)
Y = []  # Loop through each pair (X_values, labels) in the input_image dataset

for X_values, labels in input_image:
    X.append(X_values)
    Y.append(labels)

In [83]:
# Convert the list of features (X) into a NumPy array for efficient numerical operations
X = np.array(X)

# Convert the list of labels (Y) into a NumPy array for easier handling in machine learning algorithms
Y = np.array(Y)

In [85]:
# Splits the input_image dataset into training and testing subsets
train = input_image[0:4791]

test = input_image[4792:5988]

In [93]:
# Initialize empty lists to store training features (X_train) and training labels (Y_train)

X_train = []
Y_train = []

# Loop through each pair (X_values, labels) in the train dataset
for X_values, labels in train:
    
    X_train.append(X_values) # Append the feature values (X_values) to the X_train list
    
    Y_train.append(labels)  # Append the corresponding label to the Y_train list

In [95]:
# Initialize empty lists to store testing features (X_test) and testing labels (Y_test)
X_test = []
Y_test = []

# Loop through each pair (X_values, labels) in the test dataset
for X_values, labels in test:
    X_test.append(X_values)
    Y_test.append(labels)

In [97]:
# Convert the list of training features (X_train) into a NumPy array for efficient numerical operations
X_train = np.array(X_train)

# Convert the list of training labels (Y_train) into a NumPy array for easier handling in machine learning models
Y_train = np.array(Y_train)

# Convert the list of testing features (X_test) into a NumPy array for efficient numerical operations
X_test = np.array(X_test)

In [99]:
# Normalize the pixel values of the training set by dividing each pixel by 255 (scaling to the range [0, 1])
X_train = X_train / 255

# Normalize the pixel values of the testing set by dividing each pixel by 255 (scaling to the range [0, 1])
X_test = X_test / 255

In [101]:
# Initialize a Sequential model
model = Sequential()

# First Convolutional Layer: 32 filters with 5x5 kernel, ReLU activation, 'same' padding
model.add(Conv2D(filters = 32, kernel_size =(5,5), activation = 'relu', padding = 'same',))
model.add(MaxPool2D(pool_size = (2,2)))  # First MaxPooling Layer: 2x2 pool size

# Second Convolutional Layer: 32 filters with 5x5 kernel, ReLU activation
model.add(Conv2D(filters = 32, kernel_size =(5,5), activation = 'relu')) 
model.add(MaxPool2D(pool_size = (2,2))) # Second MaxPooling Layer: 2x2 pool size

# Flatten the 2D output to a 1D vector for the fully connected layers
model.add(Flatten())


# Fully Connected Layer: 128 units with ReLU activation, input shape based on X_train
model.add(Dense(128, activation = 'relu', input_shape = X_train.shape[1:]))
model.add(Dense(2, activation = 'softmax'))  # Output Layer: 2 units (for binary classification), Softmax activation to give probabilities


C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [103]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])  # Compiling the model with the Adam optimizer, sparse categorical cross-entropy loss, and accuracy metric

In [105]:
model.fit(X_train, Y_train, batch_size=64, epochs = 10, validation_split= .1) # Train the model on the training data with the specified batch size, number of epochs, and validation split

Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 73s 916ms/step - accuracy: 0.7451 - loss: 0.8610 - val_accuracy: 0.6542 - val_loss: 0.7706
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 56s 827ms/step - accuracy: 0.9367 - loss: 0.1678 - val_accuracy: 0.8729 - val_loss: 0.4022
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 53s 785ms/step - accuracy: 0.9546 - loss: 0.1264 - val_accuracy: 0.8250 - val_loss: 0.4342
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 56s 825ms/step - accuracy: 0.9660 - loss: 0.0934 - val_accuracy: 0.9750 - val_loss: 0.0915
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 60s 878ms/step - accuracy: 0.9703 - loss: 0.0841 - val_accuracy: 0.9021 - val_loss: 0.3271
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 62s 907ms/step - accuracy: 0.9537 - loss: 0.1348 - val_accuracy: 0.9375 - val_loss: 0.1641
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.9756 - loss: 0.0677 - val_accuracy: 0.8979 - val_loss: 0.2665
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 67s 977ms/step - accuracy: 0.9803 - loss: 0.0496 - val_accurac

In [117]:
# Use the trained model to predict the output (class probabilities) for the test set
pred_test = model.predict(X_test)

38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step


In [119]:
pred_test_cate = pred_test.argmax(axis = 1)  # Convert predicted probabilities to class labels by selecting the index of the highest probability

In [121]:
from sklearn.metrics import confusion_matrix, classification_report

In [123]:
tab = confusion_matrix(Y_test, pred_test_cate)  # Compute the confusion matrix to compare predicted vs. true labels
print(classification_report(Y_test, pred_test_cate))  # Print the classification report with precision, recall, F1-score, and support for each class

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.97      0.99      1196

    accuracy                           0.97      1196
   macro avg       0.50      0.49      0.49      1196
weighted avg       1.00      0.97      0.99      1196



C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [125]:
tab.diagonal().sum()/tab.sum() # Calculate the accuracy by dividing the sum of correct predictions (diagonal elements) by the total number of predictions

0.9732441471571907

In [127]:
model.save('Mask_NoMask_model.h5') # Saving the model

In [129]:
# This code captures video from a webcam, detects faces using a Haar Cascade classifier,
# processes the faces through a pre-trained CNN model to predict if a mask is present, 
# and displays the results with labeled rectangles around the faces.

from PIL import Image

model=load_model("Mask_NoMask_model.h5")

results={0:'Mask found',1:'Mask not found '}
GR_dict={0:(0,255,0),1:(0,0,255)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(r"C:\Users\ASUS\OneDrive\Desktop\Imarticus AI and ML\Haarcascade\haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f]

        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(200,200)) #  same val as used in cnn
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3)) #  same val as used in cnn
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('Liv Camera',   im)
    key = cv2.waitKey(10)
    
    if key == 27: # use the escape key
        break

cap.release()

cv2.destroyAllWindows()
        

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━